In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import os

In [ ]:
""" DEFAULTS:
# Set constants

# How often to evaluate the model
eval_interval = 20

# Number of training iterations
max_iters = 5000

learning_rate = 3e-4

batch_size = 128

# Max sequence length
block_size = 256

# Embedding size
n_embd = 256

# Number of heads in the multi-head attention
n_head = 8

# Number of blocks in the network
n_layer = 10

dropout = 0.2

# Scheduled sampling parameters
initial_teacher_forcing_prob = 1.0
final_teacher_forcing_prob = 0.5
decay_steps = max_iters
"""

# Set constants

# How often to evaluate the model
eval_interval = 20

# Number of training iterations
max_iters = 5000

learning_rate = 3e-4

batch_size = 128

# Max sequence length
block_size = 256

# Embedding size
n_embd = 256

# Number of heads in the multi-head attention
n_head = 8

# Number of blocks in the network
n_layer = 10

dropout = 0.2

# Scheduled sampling parameters
initial_teacher_forcing_prob = 1.0
final_teacher_forcing_prob = 0.5
decay_steps = max_iters

In [ ]:
class Head(nn.Module):
    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B, T, C = x.shape
        k = self.key(x)
        q = self.query(x)
        v = self.value(x)
        wei = q @ k.transpose(-2, -1)
        wei *= C**-0.5
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf'))
        wei = F.softmax(wei, dim=-1)
        wei = self.dropout(wei)
        out = wei @ v
        return out

class MultiHeadAttention(nn.Module):
    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(n_embd, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([head(x) for head in self.heads], dim=-1)
        out = self.proj(out)
        out = self.dropout(out)
        return out

class FeedForward(nn.Module):
    def __init__(self, n_embd=n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, n_embd*4),
            nn.ReLU(),
            nn.Linear(n_embd*4, n_embd),
            nn.Dropout(dropout)
        )

    def forward(self, x):
        return self.net(x)

class Block(nn.Module):
    def __init__(self, n_embd, n_head):
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedForward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

class GPTlite(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head) for _ in range(n_layer)])
        self.ln = nn.LayerNorm(n_embd)
        self.lm_head = nn.Linear(n_embd, vocab_size, bias=False)

    def forward(self, idx):
        B, T = idx.shape
        tok_emb = self.token_embedding_table(idx)
        pos_emb = self.position_embedding_table(torch.arange(T, device=idx.device))
        x = tok_emb + pos_emb
        x = self.blocks(x)
        x = self.ln(x)
        logits = self.lm_head(x)
        logits = torch.swapaxes(logits, 1, 2)
        return logits

    def generate(self, idx, max_new_tokens, temp=1.0):
        for _ in range(max_new_tokens):
            idx_cond = idx[:, -block_size:]
            logits = self(idx_cond)
            logits = logits[:, :, -1]
            logits /= temp
            probs = F.softmax(logits, dim=-1)
            idx_next = torch.multinomial(probs, num_samples=1)
            idx = torch.cat([idx, idx_next], dim=-1)
            yield idx_next.item()

In [ ]:
def scheduled_sampling_prob(step, initial_prob, final_prob, decay_steps):
    return max(final_prob, initial_prob - (step / decay_steps) * (initial_prob - final_prob))

torch.manual_seed(42)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# device = torch.device("cpu")

In [ ]:
if not os.path.exists("data"):
    os.makedirs("data")
    os.system(
        "wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt -O data/input.txt"
    )

with open("data/input.txt", "r") as f:
    text = f.read()

In [ ]:
chars = sorted(list(set(text)))
stoi = {ch: i for i, ch in enumerate(chars)}
itos = {i: ch for i, ch in enumerate(chars)}

encode = lambda x: torch.tensor([stoi[ch] for ch in x], dtype=torch.long)
decode = lambda x: "".join([itos[i] for i in x])
vocab_size = len(stoi)

data = encode(text)
n = int(0.9 * len(data))
train_data, valid_data = data[:n], data[n:]

def get_batch(source):
    ix = torch.randint(len(source) - block_size, (batch_size,))
    x = torch.stack([source[i : i + block_size] for i in ix])
    y = torch.stack([source[i + 1 : i + 1 + block_size] for i in ix])
    return x.to(device), y.to(device)

In [ ]:
m = GPTlite(vocab_size).to(device)

# number of parameters
print(f"{sum(p.numel() for p in m.parameters()):,} parameters")

# memory footprint
print(f"{sum(p.numel() * p.element_size() for p in m.parameters()) / 1024 / 1024:.1f} MB")

In [ ]:
SAVE_PATH = "data/weights2.pt"
if os.path.exists(SAVE_PATH):
    print("loading model")
    m.load_state_dict(torch.load(SAVE_PATH))
else:
    optimizer = torch.optim.Adam(m.parameters(), lr=learning_rate)
    for steps in range(max_iters):
        idx, targets = get_batch(train_data)
        # print(idx.shape, targets.shape) # This is supposed to be (batch_size, block_size) for both
        sampling_prob = scheduled_sampling_prob(steps, initial_teacher_forcing_prob, final_teacher_forcing_prob, decay_steps)
        inputs = idx
        if torch.rand(1).item() < sampling_prob:
            inputs = idx
        else:
            inputs = torch.cat([idx[:, :-1], torch.zeros(batch_size, 1, dtype=torch.long, device=device)], dim=-1)
            # print(inputs.shape)
            # raise Exception("stop here")
        # inputs = idx
        logits = m(inputs)
        loss = F.cross_entropy(logits.reshape(-1, vocab_size), targets.view(-1))
        
        loss.backward()
        optimizer.step()
        optimizer.zero_grad(set_to_none=True)
        
        if steps % 100 == 0: 
            print(f"step {steps}, loss {loss.item():.2f}")

        @torch.no_grad()
        def eval_loss():
            idx, targets = get_batch(valid_data)
            logits = m(idx)
            loss = F.cross_entropy(logits.reshape(-1, vocab_size), targets.view(-1))
            print(f"step {steps}, eval loss {loss.item():.2f}")
            return loss

        if steps % eval_interval == 0: eval_loss().item()

    print("saving model")
    torch.save(m.state_dict(), SAVE_PATH)

In [ ]:
def generate_text(start_text, max_new_tokens=100):
    idx = encode(start_text)[None, :].to(device)
    print(start_text, end='', flush=True)
    for token in m.generate(idx, max_new_tokens=max_new_tokens):
        next_char = decode([token])
        print(next_char, end='', flush=True)

In [ ]:
generate_text("ROMEO: ", max_new_tokens=1000)